Download data

In [1]:
import requests
import os
import json
import sys
os.chdir('..')
sys.path.append('..')


# Definisci la tua query Overpass. Questo esempio recupera i nodi con accessibilità per sedie a rotelle
# nell'area specificata. Modifica le coordinate dell'area secondo necessità.
overpass_url = "http://overpass-api.de/api/interpreter"
#Milano
overpass_query = """
[out:json];
(
  node["wheelchair"="yes"](around:10000,45.4641,9.1919);
  node["wheelchair"="limited"](around:10000,45.4641,9.1919);
  node["wheelchair"="no"](around:10000,45.4641,9.1919);
);
out body;
"""
#Padova
overpass_query = """
[out:json];
(
  node["wheelchair"="yes"](around:10000,45.4064,11.8768);  
  node["wheelchair"="limited"](around:10000,45.4064,11.8768);
  node["wheelchair"="no"](around:10000,45.4064,11.8768);
);
out body;
"""
# Esegui la query e ottieni i dati
response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# Salva i dati in un file GeoJSON (modifica il percorso del file come necessario)
output_geojson_path = "data/test_data.geojson"
with open(output_geojson_path, "w") as f:
    json.dump(data, f)


In [3]:
# Importa le librerie necessarie
from qgis.core import QgsVectorLayer, QgsProject, QgsExpression, QgsCategorizedSymbolRenderer, QgsSymbol

# Carica il layer OSM (ad esempio, un file GeoJSON ottenuto da OSM)
osm_path = 'path/to/your_osm_data.osm'  # Modifica con il percorso effettivo del file
vlayer = QgsVectorLayer(osm_path, "OSM Data", "ogr")
if not vlayer.isValid():
    print("Layer failed to load!")
else:
    QgsProject.instance().addMapLayer(vlayer)

# Filtra i nodi basandoti sui tag 'wheelchair'
expression = "(\"wheelchair\" = 'yes' OR \"wheelchair\" = 'limited' OR \"wheelchair\" = 'no')"
vlayer.setSubsetString(expression)

# Crea simboli per ogni categoria
symbol_yes = QgsSymbol.defaultSymbol(vlayer.geometryType())
symbol_yes.setColor(QColor("green"))
symbol_yes.setOpacity(0.5)

symbol_limited = QgsSymbol.defaultSymbol(vlayer.geometryType())
symbol_limited.setColor(QColor("yellow"))
symbol_limited.setOpacity(0.5)

symbol_no = QgsSymbol.defaultSymbol(vlayer.geometryType())
symbol_no.setColor(QColor("red"))
symbol_no.setOpacity(0.5)

# Assegna i simboli alle categorie
categories = []
for value, symbol in [("yes", symbol_yes), ("limited", symbol_limited), ("no", symbol_no)]:
    expression = "\"wheelchair\" = '%s'" % value
    category = QgsRendererCategory(expression, symbol, value)
    categories.append(category)

# Applica il renderer categorico al layer
renderer = QgsCategorizedSymbolRenderer(None, categories)
renderer.setClassAttribute("wheelchair")
vlayer.setRenderer(renderer)

# Aggiorna il layer
vlayer.triggerRepaint()


<Response [400]>